In [3]:
import numpy as np
%pylab
%matplotlib inline

from Functions import functions as f
import skimage.io as sk



Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


ImportError: cannot import name 'functions' from 'Functions' (/Users/johannamoeller/PycharmProjects/2021-topic-04-team-04/Functions/__init__.py)

In [ ]:
img = sk.imread("../Data/N2DH-GOWT1/img/t01.tif") #Bild laden
print (img.shape)

In [ ]:
img_n = f.img_resize(img,400,400)
f.show_image(img_n,15,8)

In [ ]:
t = np.zeros((5,5))

In [ ]:
from PIL import Image


test = np.zeros([100,100])
test[30:40,30:40] = 1
f.show_image(test,15,8)

im = Image.fromarray(test)
im.save("unittest.tif")